In [ ]:
import sys
#sys.path.append('./codes/forgraph/')
from config import args
from sklearn.metrics import roc_auc_score
from models import GCN
from metrics import *
import numpy as np
from Explainer import Explainer
from scipy.sparse import coo_matrix,csr_matrix
import pickle as pkl
import csv
import pandas as pd
import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

In [ ]:
def acc(adj,insert):
    mask = explainer.masked_adj.numpy()
    adj = coo_matrix(adj)
    for r,c in list(zip(adj.row,adj.col)):
        if r>=insert and r<insert+skip and c>=insert and c<insert+skip:
            reals.append(1)
        else:
            reals.append(0)
        preds.append(mask[r][c])
        
def explain_graph(gid):
    fea,emb,adj,label,graphid = features[gid], embs[gid], adjs[gid], labels[gid], gid
    explainer((fea,emb,adj,1.0,label))
    insert = 20
    acc(adj,insert)
    
def test():
    global preds
    global reals
    preds = []
    reals = []
    for gid in allnodes:
        explain_graph(gid)
    auc = roc_auc_score(reals,preds)
    return auc

def train(epochs):
    t0 = args.coff_t0
    t1 = args.coff_te

    for epoch in range(epochs): 
        loss = 0
        tmp = float(t0 * np.power(t1 / t0, epoch /epochs))
        train_instances = [ins for ins in range(adjs.shape[0])]
        np.random.shuffle(train_instances)
        for gid in train_instances:
            with tf.GradientTape() as tape:
                pred = explainer((features[gid],embs[gid],adjs[gid],tmp, labels[gid]),training=True)
                loss += explainer.loss(pred, labels[gid])
        train_variables = [para for para in explainer.trainable_variables]
        grads = tape.gradient(loss, train_variables)
        optimizer.apply_gradients(zip(grads, train_variables))

In [ ]:
file = 
with open(file,'rb') as fin:
    adjs,features,labels = pkl.load(fin)

model = GCN(input_dim=features.shape[-1], output_dim=labels.shape[1])
model.load_weights(args.save_path+args.dataset)


embs = model.getNodeEmb((tf.convert_to_tensor(features,dtype=tf.float32),\
                               tf.convert_to_tensor(adjs,dtype=tf.float32)), training=False)
outputs = model.call((tf.convert_to_tensor(features,dtype=tf.float32),\
                            tf.convert_to_tensor(adjs,dtype=tf.float32)),training=False)
pred_label = tf.argmax(outputs, 1)

acc = accuracy(outputs, labels)
print('model acc',acc.numpy())

embs = model.getNodeEmb((tf.convert_to_tensor(features,dtype=tf.float32),\
                               tf.convert_to_tensor(adjs,dtype=tf.float32)), training=False)
outputs = model.call((tf.convert_to_tensor(features,dtype=tf.float32),\
                            tf.convert_to_tensor(adjs,dtype=tf.float32)),training=False)
pred_label = tf.argmax(outputs, 1)

In [ ]:
if args.setting==1:
    allnodes = [i for i in range(0,100)]
elif args.setting==2:
    allnodes = [i for i in range(0,100)]
    allnodes.extend([i for i in range(500,600)])
elif args.setting==3:
    allnodes=[i for i in range(1000)]
explainer = Explainer(model=model,nodesize=adjs.shape[1])

In [ ]:
top_k = [i for in in range(0, 25)]
all_elr = 
coff_t0 = 
coff_size = 
coff_ent = 
skp = 
epochs = 500

In [ ]:
rows = []

In [ ]:
for k in top_k:
    topk = k
    for elr in all_elr:
        args.elr = elr
        optimizer = tf.keras.optimizers.Adam(learning_rate=args.elr)
        for c in coff_t0:
            args.coff_t0 = c
            for s in coff_size:
                args.coff_size = s
                for en in coff_ent:
                    args.coff_ent = en
                    for sk in skp:
                        skip = sk
                        for ep in epochs:
                            train(ep)
                            auc = test()
                            row = [k, elr, c, en, sk, ep, auc]
                            rows.append(row)

In [ ]:
col_names = ['K', 'elr', 'coff_t0', 'coff_size', 'coff_ent', 'Skip', 'AUC']
filename = 'results.csv'
with open('results.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerow(col_names) 
    write.writerows(rows) 

In [ ]:
df = pd.read_csv(file_name)
df.head()

In [ ]:
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label="K",
    y_axis_label="AUC",
)
bokeh.io.show(p)

In [ ]:
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label="elr",
    y_axis_label="AUC",
)
bokeh.io.show(p)

In [ ]:
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label="coff_t0",
    y_axis_label="AUC",
)
bokeh.io.show(p)

In [ ]:
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label="coff_size",
    y_axis_label="AUC",
)
bokeh.io.show(p)

In [ ]:
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label="coff_ent",
    y_axis_label="AUC",
)
bokeh.io.show(p)

In [ ]:
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label="Skip",
    y_axis_label="AUC",
)
bokeh.io.show(p)